In [36]:
import pandas as pd
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [5]:
df = pd.read_csv('oceans_12Reviews.csv')

,Unnamed: 0,ratings,reviews,usefulness_rating,name,id,reviews_clean
0,2258,7,"If this movie has a downside, and many of the ...",0.909091,Ocean's Twelve,tt0349903,if this movie has a downside and many of the r...


In [8]:
def give_sentiment(rating):
    if(rating > 5):
        return 'Positive'
    elif (rating == 5):
        return 'Neutral'
    else:
        return 'Negative'

In [9]:
df['overall_sentiment'] = df['ratings'].apply(lambda x: give_sentiment(x))

Pre Trained VADER sentiment classification

In [10]:
def vader_sentiment(text):
    analyser = SentimentIntensityAnalyzer()
    sentiment_score = analyser.polarity_scores(text)
    if(sentiment_score['compound'] > 0):
        return 'Positive'
    elif(sentiment_score['compound'] == 0):
        return 'Neutral'
    else:
        return 'Negative'

In [18]:
df['vader_sentiment'] = df['reviews_clean'].apply(lambda x: vader_sentiment(x))


In [19]:
print('Proportion of incorrectly classified reviews: ' + str(df[df['overall_sentiment'] != df['vader_sentiment']].size/df.size))

Proportion of incorrectly classified reviews: 0.516


Dropping stopwords

In [30]:
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    return ' '.join([w for w in nltk.word_tokenize(text) if w not in stopwords])


In [31]:
df['tokenized_no_stopwords_reviews'] = df['reviews_clean'].apply(lambda x: remove_stopwords(x))
df['vader_stopwords_classification'] = df['tokenized_no_stopwords_reviews'].apply(
    lambda x: vader_sentiment(x))
print('Proportion of incorrectly classified reviews: ' +
      str(df[df['overall_sentiment'] != df['vader_stopwords_classification']].size/df.size))


Proportion of incorrectly classified reviews: 0.636


In [32]:
cv = CountVectorizer()
text_vectorized = cv.fit_transform(df['tokenized_no_stopwords_reviews'])
xtrain, xtest, ytrain, ytest = train_test_split(text_vectorized, df['overall_sentiment'], test_size = 0.2, random_state=42)


In [35]:
mnb = MultinomialNB()
mnb.fit(xtrain, ytrain)
pred = mnb.predict(xtest)
accuracy_score = metrics.accuracy_score(pred, ytest)

NameError: name 'metrics' is not defined